## Get Workspace

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace.create(name='aml1p-ml-wus2', subscription_id='48bbc269-ce89-4f6f-9a12-c6f91fcb772d', resource_group='aml1p-rg')

## Distribute Remotely

In [13]:
from azureml.core import ScriptRunConfig, Experiment, Environment
from azureml.core.runconfig import MpiConfiguration

# arguments
filename = "azureml/a59853eb-a1ae-4444-8aed-6712f77f6aba/LGBMTrainTSV"
boosting = "gbdt"
iterations = 100
lr = 0.2
leaves = 31

# azureml setup
compute_target = "cpu-m64-lowpri"
compute_target = "cpu-m16ms-lowpri"

nodes = 10
cpus = 16

# submit run
arguments = ["--filename", filename, "--boosting", boosting, "--num_iterations", iterations, "--learning_rate", lr, "--num_leaves", leaves, "--nodes", nodes, "--cpus", cpus]
env = Environment.from_conda_specification("lightgbm-cpu-tutorial", "environment.yml")
mpi_config = MpiConfiguration(node_count=nodes)
src = ScriptRunConfig(
    source_directory="src",
    script="run.py",
    arguments=arguments,
    compute_target=compute_target,
    environment=env,
    distributed_job_config=mpi_config,
)
run = Experiment(ws, "cody-lightgbm-dask-test").submit(src)
run

In [ ]:
run 

In [12]:
run.cancel()